# Access to Fraction of Absorbed Photosynthetically Active Radiation
**Downloading, Reading and Working with Copernicus FAPAR** 


## Overview


## Imports

In [ ]:
import os

import cartopy.crs as ccrs
import hvplot.xarray  # noqa
import rioxarray  # noqa
import xarray as xr
from dotenv import dotenv_values
from hda import Client, Configuration

In [ ]:
conf = Configuration(
    user=dotenv_values(".env")["USER_WEKEO"],
    password=dotenv_values(".env")["PASS_WEKEO"],
)
hda_client = Client(config=conf)

In [ ]:
query = {
    "dataset_id": "EO:CLMS:DAT:CLMS_GLOBAL_FAPAR_300M_V1_10DAILY_NETCDF",
    "productGroupId": "RT6",
    "bbox": [
        32.9,
        -18.1,
        34.4,
        -19.6,
    ],
    "startdate": "2025-07-10T00:00:00.000Z",
    "enddate": "2025-07-11T00:59:59.999Z",
    "itemsPerPage": 200,
    "startIndex": 0,
}

matches = hda_client.search(query)

print(matches)

In [ ]:
%%capture

local_path = "cgls_fapar_300"

if not os.path.isdir(local_path):
    os.mkdir(local_path)

matches.download(download_dir=local_path)

In [ ]:
ds = (
    xr.open_mfdataset(
        "cgls_fapar_300/*.nc",
        combine="nested",
        concat_dim="time",
        parallel=True,
        chunks={"time": 1, "lat": 3000, "lon": 3000},
    )
    .rio.write_crs("EPSG:4326")
    .rio.clip_box(
        32.9,
        -19.6,
        34.4,
        -18.1,
    )
)
ds

In [ ]:
ds.FAPAR.hvplot.image(
    x="lon",
    y="lat",
    rasterize=True,
    crs=ccrs.PlateCarree(),
    tiles=True,
    frame_width=500,
    clabel="Fraction of Absorbed Photosynthetically Active Radiation",
)